In [50]:
import numpy as np
import pandas as pd
import darts
from darts import TimeSeries
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
#import seaborn as sns
from datetime import datetime
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, r2_score, rmse

from darts.models import (
    AutoARIMA,
    Prophet,
    RNNModel,
    NBEATSModel,
    BlockRNNModel,
    TFTModel,
)

from tqdm import tqdm
import numpy as np

In [51]:
df = pd.read_csv('../input/rossmann-store-sales/train.csv', parse_dates=['Date'])
df.sample(10)
df = pd.concat([df.drop(columns='StateHoliday'), pd.get_dummies(df.StateHoliday, prefix='Holiday')], axis=1)

In [52]:
store = pd.read_csv('../input/rossmann-store-sales/store.csv')

In [53]:
def get_sample(num):
    sample = df[df.Store == num]
    series = TimeSeries.from_dataframe(sample, 'Date', 'Sales')

    train, test = series.split_before(pd.Timestamp("20150601"))

    transformer = Scaler()
    train_transformed = transformer.fit_transform(train)
    test_transformed = transformer.transform(test)

    series_customers = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Customers')
    series_open = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Open')
    series_promo = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Promo')
    series_school = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='SchoolHoliday')
    series_weekday = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='DayOfWeek')
    series_holiday_a = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Holiday_a')
    series_holiday_b = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Holiday_b')
    series_holiday_c = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Holiday_c')

    customers_transformed = transformer.transform(series_customers)

    covariates = series_customers.stack(series_open)
    covariates = covariates.stack(series_promo)
    covariates = covariates.stack(series_school)
    covariates = covariates.stack(series_weekday)
    covariates = covariates.stack(series_holiday_a)
    covariates = covariates.stack(series_holiday_b)
    covariates = covariates.stack(series_holiday_c)

    train_covariates, test_covariates = covariates.split_before(pd.Timestamp("20150601"))

    return train_transformed, test_transformed, covariates

In [54]:
train, test, covariates = get_sample(10)

In [ ]:
cdf = df.groupby('Store').count().reset_index()[['Store', 'Date']]
store_merged = pd.merge(store, cdf, left_on='Store', right_on='Store')
store_merged = store_merged[store_merged.Date == 942]

In [55]:
def rmspe(actual, pred):
    return np.sqrt(np.mean( ((actual - pred) / actual)**2)) 

def smape(actual, pred):
    return np.mean(np.abs(pred - actual) / ((np.abs(actual) + np.abs(pred))/2)) * 100

def evaluate_model(model, train, test):
    model.fit(train)
    pred = model.predict(len(test))
    test = test.pd_dataframe()
    pred = pred.pd_dataframe()

    test.columns = ['Actual']
    pred.columns = ['Pred']

    df = pd.concat([test, pred], axis=1)
    df = df[df.Actual > 0]
    return smape(df.Actual.values, df.Pred.values), rmspe(df.Actual.values, df.Pred.values)

In [57]:
train_transformed, test_transformed, covariates = get_sample(10)
#nn_model = RNNModel(input_chunk_length=30, output_chunk_length=10)
#rmspe_score, smape_score = evaluate_model(nn_model, train_transformed, test_transformed )
#rmspe_score, smape_score

In [58]:
store_num_list = pd.read_csv('../input/store_list.csv').Store.values

## 1. LSTM

### 1.1 Naive LSTM Model

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.05,
    mode='min',
)

In [ ]:
smape_list = []
rmspe_list = []

from darts.metrics.metrics import mape
for num in tqdm(store_num_list):
    train_transformed, test_transformed, covariates = get_sample(num)
    rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout=0.3,
            batch_size=16,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )

    smape_score, rmspe_score = evaluate_model(rnn_model, train_transformed, test_transformed )
    rmspe_list.append(rmspe_score)
    smape_list.append(smape_score)

In [ ]:
train_covariates, test_covariates = covariates.split_before(pd.Timestamp("20150601"))

rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout=0.3,
            batch_size=16,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )
rnn_model.fit(train_transformed)
pred_demand_covs = rnn_model.predict(len(test_transformed))

plt.figure(figsize=(15, 6))
test_transformed.plot(label="actual")
pred_demand_covs.plot(label="forecast")
plt.title('RNN')
plt.show()

In [ ]:
train_covariates, test_covariates = covariates.split_before(pd.Timestamp("20150601"))

rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout=0.3,
            batch_size=16,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )
rnn_model.fit(train_transformed, future_covariates=covariates)
pred_demand_covs = rnn_model.predict(len(test_transformed), future_covariates=covariates)

plt.figure(figsize=(15, 6))
test_transformed.plot(label="actual")
pred_demand_covs.plot(label="forecast")
plt.title('RNN with Covariates')
plt.show()

In [ ]:
print(np.mean(smape_list), np.std(smape_list), np.max(smape_list), np.min(smape_list))
plt.hist(smape_list)

In [ ]:
print(np.mean(rmspe_list), np.std(rmspe_list), np.max(rmspe_list), np.min(rmspe_list))
plt.hist(rmspe_list)

In [ ]:
res1 = pd.DataFrame(
    {
        'rmspe' : rmspe_list,
        'smape' : smape_list,
    }
)

In [ ]:
res1.mean()

### 1.2 Covrariate LSTM Model

In [60]:
def evaluate_covariates_model(model, train, test, covariates):
    model.fit(
        train,
        future_covariates=covariates,
        verbose=False,
    )
    pred = model.predict(len(test), future_covariates=covariates)
    pred = pred.pd_dataframe()
    test = test.pd_dataframe()

    test.columns = ['Actual']
    pred.columns = ['Pred']

    df = pd.concat([test, pred], axis=1)
    df = df[df.Actual > 0]
    return smape(df.Actual.values, df.Pred.values), rmspe(df.Actual.values, df.Pred.values)

In [ ]:
smape_list = []
rmspe_list = []

for num in tqdm(store_num_list):
    train_transformed, test_transformed, covariates = get_sample(num)
    rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout=0.3,
            batch_size=16,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )
    mape_score, rmse_score = evaluate_covariates_model(rnn_model, train_transformed, test_transformed, covariates )
    smape_list.append(mape_score)
    rmspe_list.append(rmse_score)


In [ ]:
print(np.mean(smape_list), np.std(smape_list), np.max(smape_list), np.min(smape_list))
plt.hist(smape_list)

In [ ]:
print(np.mean(rmspe_list), np.std(rmspe_list), np.max(rmspe_list), np.min(rmspe_list))
plt.hist(rmspe_list)

In [ ]:
res1['Store'] = store_num_list

### 1.3 Probabilistic LSTM - DeepAR

In [ ]:
from darts.utils.likelihood_models import GaussianLikelihood

In [ ]:
smape_list = []
rmspe_list = []

from darts.metrics.metrics import mape
for num in tqdm(store_num_list):

    train_transformed, test_transformed, covariates = get_sample(num)
    rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout=0.3,
            batch_size=16,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            likelihood=GaussianLikelihood(),
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )

    mape_score, rmse_score = evaluate_covariates_model(rnn_model, train_transformed, test_transformed, covariates )
    smape_list.append(mape_score)
    rmspe_list.append(rmse_score)

In [ ]:
print(np.mean(smape_list), np.std(smape_list), np.max(smape_list), np.min(smape_list))
plt.hist(smape_list)

In [ ]:
print(np.mean(rmspe_list), np.std(rmspe_list), np.max(rmspe_list), np.min(rmspe_list))
plt.hist(rmspe_list)

In [ ]:
res3 = pd.DataFrame(
    {
        'rmspe' : rmspe_list,
        'smape' : smape_list,
    }
)

In [ ]:
train_covariates, test_covariates = covariates.split_before(pd.Timestamp("20150601"))

rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout=0.3,
            batch_size=16,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            likelihood=GaussianLikelihood(),
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )
rnn_model.fit(train_transformed, future_covariates=covariates)
pred_demand_covs = rnn_model.predict(len(test_transformed), future_covariates=covariates)

plt.figure(figsize=(15, 6))
test_transformed.plot(label="actual")
pred_demand_covs.plot(label="forecast")
plt.title('DeepAR')
plt.show()

### 1.5 Result Comparison

In [ ]:
res1['Store'] = store_num_list
res2['Store'] = store_num_list
res3['Store'] = store_num_list


result = pd.merge(res1, res2, left_on='Store', right_on='Store')
result = pd.merge(result, res3, left_on='Store', right_on='Store')
result = result.set_index('Store')
result = pd.merge(result, store[['Store','StoreType']], left_on='Store', right_on='Store')
result.to_csv('../output/rnn_result.csv')
result.mean()

### Hyperparameter Tunning

In [26]:
import pandas as pd
from pytorch_lightning.callbacks import EarlyStopping
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection

In [46]:
def train_model(model_args, callbacks, train, val, covariates):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    # Create the model using model_args from Ray Tune
    model = RNNModel(
            model="LSTM",
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            torch_metrics=torch_metrics,
            pl_trainer_kwargs = {"callbacks": callbacks, "enable_progress_bar": False},
        **model_args)

    model.fit(
        series=train,
        val_series=val,
        future_covariates=covariates,
        val_future_covariates=covariates
    )

In [47]:
train, test, covariates = get_sample(10)
train, val = train.split_after(pd.Timestamp(year=2015, month=4, day=1))

In [48]:
my_stopper = EarlyStopping(
    monitor="val_MeanAbsolutePercentageError",
    patience=5,
    min_delta=0.05,
    mode='min',
)

# set up ray tune callback
tune_callback = TuneReportCallback(
    {
        "loss": "val_Loss",
        "MAPE": "val_MeanAbsolutePercentageError",
    },
    on="validation_end",
)

# define the hyperparameter space
config = {
    "batch_size": tune.choice([16, 32, 64, 128]),
    "hidden_dim": tune.choice([30, 60, 90]),
    "dropout": tune.uniform(0, 0.3),
}

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

resources_per_trial = {"cpu": 16, "gpu": 1}

# the number of combinations to try
num_samples = 10

scheduler = ASHAScheduler(max_t=1000, grace_period=3, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train, val=val, covariates=covariates
)

# optimize hyperparameters by minimizing the MAPE on the validation set
analysis = tune.run(
    train_fn_with_parameters,
    resources_per_trial=resources_per_trial,
    # Using a metric instead of loss allows for
    # comparison between different likelihood or loss functions.
    metric="MAPE",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

== Status ==
Current time: 2023-01-30 23:00:08 (running for 00:00:00.20)
Memory usage on this node: 15.9/31.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+--------------+-----------+
| Trial name              | status   | loc             |   batch_size |   hidden_dim |   dropout |
|-------------------------+----------+-----------------+--------------+--------------+-----------|
| train_model_c9a62_00000 | RUNNING  | 127.0.0.1:12020 |           16 |           60 | 0.251974  |
| train_model_c9a62_00001 | PENDING  |                 |           16 |       

(train_model pid=12020) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.251973683175626 and num_layers=1
(train_model pid=12020)   warnings.warn("dropout option adds dropout after all but last "
(train_model pid=12020) GPU available: True (cuda), used: False
(train_model pid=12020) TPU available: False, using: 0 TPU cores
(train_model pid=12020) IPU available: False, using: 0 IPUs
(train_model pid=12020) HPU available: False, using: 0 HPUs
(train_model pid=12020) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=12020)   rank_zero_warn(
(train_model pid=12020) 
(train_model pid=12020)   | Name          | Type         

Trial name,MAPE,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_c9a62_00000,83773.1,2023-01-30_23-00-17,True,,e36d4d461b5f4105bb515f48e7e5e59e,"0_batch_size=16,dropout=0.2520,hidden_dim=60",Tensortorch,6,127.0.0.1,12020,3.27175,0.504902,3.27175,1675090817,0,,6,c9a62_00000,0.00403166
train_model_c9a62_00001,83773.1,2023-01-30_23-00-25,True,,a8d8b8b2215f4ccb9ad7604ae969dc0a,"1_batch_size=16,dropout=0.2559,hidden_dim=60",Tensortorch,6,127.0.0.1,30732,3.08138,0.477228,3.08138,1675090825,0,,6,c9a62_00001,0.00506687
train_model_c9a62_00002,84218.2,2023-01-30_23-00-32,True,,beabb25f706944baaf66dbe243a7715e,,Tensortorch,3,127.0.0.1,9872,1.11676,0.326571,1.11676,1675090832,0,,3,c9a62_00002,0.0048008
train_model_c9a62_00003,90838.5,2023-01-30_23-00-40,True,,50d45a7dca6a484fbdadb09554b4c136,,Tensortorch,3,127.0.0.1,30480,1.488,0.43326,1.488,1675090840,0,,3,c9a62_00003,0.0049181
train_model_c9a62_00004,89244.7,2023-01-30_23-00-46,True,,f4b1046f949849aa8d6993695e28a697,,Tensortorch,3,127.0.0.1,17452,0.998566,0.289863,0.998566,1675090846,0,,3,c9a62_00004,0.00519109
train_model_c9a62_00005,83773.1,2023-01-30_23-00-55,True,,388ee560d0674b7ca5a8c961923a066d,"5_batch_size=16,dropout=0.2967,hidden_dim=60",Tensortorch,6,127.0.0.1,6960,3.09162,0.483547,3.09162,1675090855,0,,6,c9a62_00005,0.00353503
train_model_c9a62_00006,90838.5,2023-01-30_23-01-03,True,,d6cfdc9f5e3d4d44a1f867e389e02a9f,,Tensortorch,3,127.0.0.1,30700,1.47554,0.428732,1.47554,1675090863,0,,3,c9a62_00006,0.00416946
train_model_c9a62_00007,85308.6,2023-01-30_23-01-11,True,,bd486a797fc24f91a471d1d72f4f4fce,,Tensortorch,6,127.0.0.1,26168,3.25258,0.52614,3.25258,1675090871,0,,6,c9a62_00007,0.00572491
train_model_c9a62_00008,83675.8,2023-01-30_23-01-20,True,,c1426a461245490b8e733ebbc4bdb3cf,"8_batch_size=32,dropout=0.0061,hidden_dim=60",Tensortorch,6,127.0.0.1,32076,2.46181,0.386256,2.46181,1675090880,0,,6,c9a62_00008,0.00398135
train_model_c9a62_00009,71638.1,2023-01-30_23-01-27,True,,918bf9f049eb4d3bb6d591c1123fc121,"9_batch_size=128,dropout=0.1666,hidden_dim=60",Tensortorch,6,127.0.0.1,9976,1.9888,0.298176,1.9888,1675090887,0,,6,c9a62_00009,0.00416851


== Status ==
Current time: 2023-01-30 23:00:14 (running for 00:00:05.91)
Memory usage on this node: 16.3/31.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: c9a62_00000 with MAPE=76198.34725618408 and parameters={'batch_size': 16, 'hidden_dim': 60, 'dropout': 0.251973683175626}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+--------------+-----------+---------+----------------------+
| Trial name              | status   | loc             |   batch_size |   hidden_dim |   dropout |    MAPE |   training_iteration |
|-------------------------+----------+-----------------+------------

(train_model pid=12020) 2023-01-30 23:00:14,529	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=12020) 2023-01-30 23:00:15,064	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=12020) 2023-01-30 23:00:15,577	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=12020) 2023-01-30 23:00:16,110	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=12020) 2023-01-30 23:00:16,630	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=12020) 2023-01-30 23:00:17,135	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.


...

== Status ==
Current time: 2023-01-30 23:01:17 (running for 00:01:09.15)
Memory usage on this node: 16.2/31.6 GiB 
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: -83773.08526340287 | Iter 3.000: -84048.08211406544
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: c9a62_00000 with MAPE=83773.08526340287 and parameters={'batch_size': 16, 'hidden_dim': 60, 'dropout': 0.251973683175626}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (1 PENDING, 1 RUNNING, 8 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+-----------+---------+----------------------+
| Trial name              | status     | loc             |   batch_size |   hidden_dim |   dropout |    MAPE |   training_iteration |
|---------------------

(train_model pid=32076) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.006109897189298941 and num_layers=1
(train_model pid=32076)   warnings.warn("dropout option adds dropout after all but last "
(train_model pid=32076) GPU available: True (cuda), used: False
(train_model pid=32076) TPU available: False, using: 0 TPU cores
(train_model pid=32076) IPU available: False, using: 0 IPUs
(train_model pid=32076) HPU available: False, using: 0 HPUs
(train_model pid=32076) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=32076)   rank_zero_warn(
(train_model pid=32076) 
(train_model pid=32076)   | Name          | Type      

== Status ==
Current time: 2023-01-30 23:01:25 (running for 00:01:17.19)
Memory usage on this node: 16.2/31.6 GiB 
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: -83773.08526340287 | Iter 3.000: -83877.97106950564
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: c9a62_00008 with MAPE=83675.84334219756 and parameters={'batch_size': 32, 'hidden_dim': 60, 'dropout': 0.006109897189298941}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+-----------+---------+----------------------+
| Trial name              | status     | loc             |   batch_size |   hidden_dim |   dropout |    MAPE |   training_iteration |
|-------------------------+---

(train_model pid=9976) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.16657176761791098 and num_layers=1
(train_model pid=9976)   warnings.warn("dropout option adds dropout after all but last "
(train_model pid=9976) GPU available: True (cuda), used: False
(train_model pid=9976) TPU available: False, using: 0 TPU cores
(train_model pid=9976) IPU available: False, using: 0 IPUs
(train_model pid=9976) HPU available: False, using: 0 HPUs
(train_model pid=9976) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=9976)   rank_zero_warn(
(train_model pid=9976) 
(train_model pid=9976)   | Name          | Type             | Pa

== Status ==
Current time: 2023-01-30 23:01:27 (running for 00:01:19.25)
Memory usage on this node: 16.3/31.6 GiB 
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: -83773.08526340287 | Iter 3.000: -83382.13874869357
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: c9a62_00009 with MAPE=71638.06035203935 and parameters={'batch_size': 128, 'hidden_dim': 60, 'dropout': 0.16657176761791098}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+-----------+---------+----------------------+
| Trial name              | status     | loc             |   batch_size |   hidden_dim |   dropout |    MAPE |   training_iteration |
|-------------------------+------------+-----

In [61]:
smape_list = []
rmspe_list = []

for num in tqdm(store_num_list):
    train_transformed, test_transformed, covariates = get_sample(num)
    rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=60,
            dropout= 0.16657176761791098,
            batch_size=128,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )
    mape_score, rmse_score = evaluate_covariates_model(rnn_model, train_transformed, test_transformed, covariates )
    smape_list.append(mape_score)
    rmspe_list.append(rmse_score)


  0%|          | 0/100 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  1%|          | 1/100 [00:25<42:51, 25.98s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  2%|▏         | 2/100 [00:49<39:50, 24.

In [62]:
res_h1 = pd.DataFrame(
    {
        'rmspe' : rmspe_list,
        'smape' : smape_list,
    }
)
res_h1['Store'] = store_num_list
res_h1

,rmspe,smape,Store
0,0.086285,7.701973,10
1,0.049630,3.829394,62
2,0.052607,4.260640,110
3,0.070980,5.526770,163
4,0.060497,4.899070,300
...,...,...,...
95,0.050640,4.221876,960
96,0.079968,7.179742,1010
97,0.050098,4.260537,1046
98,0.065118,5.406682,1089


In [66]:
res_h1.mean()

rmspe      0.076045
smape      6.459042
Store    585.390000
dtype: float64

### DeepAR - Hyperparameter Tunning

In [63]:
from darts.utils.likelihood_models import GaussianLikelihood

def train_model(model_args, callbacks, train, val, covariates):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    # Create the model using model_args from Ray Tune
    model = RNNModel(
            model="LSTM",
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            torch_metrics=torch_metrics,
            likelihood=GaussianLikelihood(),
            pl_trainer_kwargs = {"callbacks": callbacks, "enable_progress_bar": False},
        **model_args)

    model.fit(
        series=train,
        val_series=val,
        future_covariates=covariates,
        val_future_covariates=covariates
    )

In [64]:
train, test, covariates = get_sample(10)
train, val = train.split_after(pd.Timestamp(year=2015, month=4, day=1))

In [65]:
my_stopper = EarlyStopping(
    monitor="val_MeanAbsolutePercentageError",
    patience=5,
    min_delta=0.05,
    mode='min',
)

# set up ray tune callback
tune_callback = TuneReportCallback(
    {
        "loss": "val_Loss",
        "MAPE": "val_MeanAbsolutePercentageError",
    },
    on="validation_end",
)

# define the hyperparameter space
config = {
    "batch_size": tune.choice([16, 32, 64, 128]),
    "hidden_dim": tune.choice([30, 60, 90]),
    "dropout": tune.uniform(0, 0.3),
}

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

resources_per_trial = {"cpu": 16, "gpu": 1}

# the number of combinations to try
num_samples = 10

scheduler = ASHAScheduler(max_t=1000, grace_period=3, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train, val=val, covariates=covariates
)

# optimize hyperparameters by minimizing the MAPE on the validation set
analysis = tune.run(
    train_fn_with_parameters,
    resources_per_trial=resources_per_trial,
    # Using a metric instead of loss allows for
    # comparison between different likelihood or loss functions.
    metric="MAPE",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

== Status ==
Current time: 2023-01-30 23:48:57 (running for 00:00:00.19)
Memory usage on this node: 17.9/31.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+--------------+-----------+
| Trial name              | status   | loc             |   batch_size |   hidden_dim |   dropout |
|-------------------------+----------+-----------------+--------------+--------------+-----------|
| train_model_9b6c5_00000 | RUNNING  | 127.0.0.1:11872 |           32 |           60 | 0.283982  |
| train_model_9b6c5_00001 | PENDING  |                 |           16 |       

(train_model pid=11872) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28398153787733454 and num_layers=1
(train_model pid=11872)   warnings.warn("dropout option adds dropout after all but last "
(train_model pid=11872) GPU available: True (cuda), used: False
(train_model pid=11872) TPU available: False, using: 0 TPU cores
(train_model pid=11872) IPU available: False, using: 0 IPUs
(train_model pid=11872) HPU available: False, using: 0 HPUs
(train_model pid=11872) c:\Users\daeky\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=11872)   rank_zero_warn(
(train_model pid=11872) 
(train_model pid=11872)   | Name          | Type       

Trial name,MAPE,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_9b6c5_00000,1246.77,2023-01-30_23-49-13,True,,7576b71a1e234bbcbfe7702282b19374,"0_batch_size=32,dropout=0.2840,hidden_dim=60",Tensortorch,21,127.0.0.1,11872,9.93027,0.458184,9.93027,1675093753,0,,21,9b6c5_00000,0.0029943
train_model_9b6c5_00001,2422.18,2023-01-30_23-49-22,True,,4d4e6d548daf4db7881c9d6a55242e64,,Tensortorch,6,127.0.0.1,32088,3.71122,0.57908,3.71122,1675093762,0,,6,9b6c5_00001,0.00359297
train_model_9b6c5_00002,21652.3,2023-01-30_23-49-29,True,,1e641dadc7cb41fb8a0cd01cf7c27909,,Tensortorch,3,127.0.0.1,34416,1.82825,0.544136,1.82825,1675093769,0,,3,9b6c5_00002,0.00513673
train_model_9b6c5_00003,16413.7,2023-01-30_23-49-36,True,,61d892dd525845a3be3b253637508479,,Tensortorch,3,127.0.0.1,476,1.39729,0.410841,1.39729,1675093776,0,,3,9b6c5_00003,0.00399899
train_model_9b6c5_00004,39777.1,2023-01-30_23-49-43,True,,b23769bec92e402b9376e2dd1a391063,,Tensortorch,3,127.0.0.1,18048,1.34503,0.387234,1.34503,1675093783,0,,3,9b6c5_00004,0.00490594
train_model_9b6c5_00005,2422.18,2023-01-30_23-49-53,True,,8b9f68fa284843cabaa1f1b995e39804,,Tensortorch,6,127.0.0.1,27488,3.88326,0.600273,3.88326,1675093793,0,,6,9b6c5_00005,0.00497699
train_model_9b6c5_00006,3841.69,2023-01-30_23-50-04,True,,6dfa104beeca403d991bcf861f53fcca,,Tensortorch,6,127.0.0.1,1548,4.90342,0.8762,4.90342,1675093804,0,,6,9b6c5_00006,0.00308251
train_model_9b6c5_00007,81424.7,2023-01-30_23-50-11,True,,9954c156fb2449cea49c83d5c3f037ec,,Tensortorch,3,127.0.0.1,32932,1.11217,0.330431,1.11217,1675093811,0,,3,9b6c5_00007,0.00402522
train_model_9b6c5_00008,81424.7,2023-01-30_23-50-18,True,,0916c18cf9104df09128bc344025199b,,Tensortorch,3,127.0.0.1,4148,1.1908,0.387702,1.1908,1675093818,0,,3,9b6c5_00008,0.0039804
train_model_9b6c5_00009,538.154,2023-01-30_23-50-40,True,,d7c429ae52db469a9fcbc4d70a7d081b,"9_batch_size=16,dropout=0.2991,hidden_dim=30",Tensortorch,28,127.0.0.1,28748,16.3461,0.518004,16.3461,1675093840,0,,28,9b6c5_00009,0.00407863


== Status ==
Current time: 2023-01-30 23:49:04 (running for 00:00:06.46)
Memory usage on this node: 18.4/31.6 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: 9b6c5_00000 with MAPE=28994.706076765517 and parameters={'batch_size': 32, 'hidden_dim': 60, 'dropout': 0.28398153787733454}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+--------------+-----------+---------+----------------------+
| Trial name              | status   | loc             |   batch_size |   hidden_dim |   dropout |    MAPE |   training_iteration |
|-------------------------+----------+-----------------+---------

(train_model pid=11872) 2023-01-30 23:49:03,998	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=11872) 2023-01-30 23:49:04,468	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=11872) 2023-01-30 23:49:04,930	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=11872) 2023-01-30 23:49:05,407	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=11872) 2023-01-30 23:49:05,860	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=11872) 2023-01-30 23:49:06,332	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=11872) 2023-01-30 23:49:06,788	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.cal

...

== Status ==
Current time: 2023-01-30 23:50:35 (running for 00:01:37.60)
Memory usage on this node: 18.4/31.6 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -1131.7847706099685 | Iter 6.000: -2422.1844725699048 | Iter 3.000: -10304.173206653199
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: 9b6c5_00009 with MAPE=661.1484879106773 and parameters={'batch_size': 16, 'hidden_dim': 30, 'dropout': 0.29909326760922184}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+-----------+-----------+----------------------+
| Trial name              | status     | loc             |   batch_size |   hidden_dim |   dropout |      MAPE |   training_iteration |
|---------

(train_model pid=28748) 2023-01-30 23:50:35,143	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:35,876	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:36,456	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:36,952	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:37,469	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:38,060	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:38,575	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.cal

== Status ==
Current time: 2023-01-30 23:50:40 (running for 00:01:42.65)
Memory usage on this node: 18.4/31.6 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: -763.5910318452559 | Iter 12.000: -1131.7847706099685 | Iter 6.000: -2422.1844725699048 | Iter 3.000: -10304.173206653199
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: 9b6c5_00009 with MAPE=552.8240614366725 and parameters={'batch_size': 16, 'hidden_dim': 30, 'dropout': 0.29909326760922184}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+-----------+-----------+----------------------+
| Trial name              | status     | loc             |   batch_size |   hidden_dim |   dropout |      MAPE |   training_iteratio

(train_model pid=28748) 2023-01-30 23:50:40,197	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=28748) 2023-01-30 23:50:40,715	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
2023-01-30 23:50:40,992	INFO tune.py:762 -- Total run time: 103.45 seconds (103.30 seconds for the tuning loop).


== Status ==
Current time: 2023-01-30 23:50:40 (running for 00:01:43.32)
Memory usage on this node: 18.4/31.6 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: -763.5910318452559 | Iter 12.000: -1131.7847706099685 | Iter 6.000: -2422.1844725699048 | Iter 3.000: -10304.173206653199
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/9.84 GiB heap, 0.0/4.92 GiB objects
Current best trial: 9b6c5_00009 with MAPE=538.154337665045 and parameters={'batch_size': 16, 'hidden_dim': 30, 'dropout': 0.29909326760922184}
Result logdir: C:\Users\daeky\ray_results\tune_darts
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+-----------------+--------------+--------------+-----------+-----------+----------------------+
| Trial name              | status     | loc             |   batch_size |   hidden_dim |   dropout |      MAPE |   training_iteration |
|-----------

In [67]:
smape_list = []
rmspe_list = []

for num in tqdm(store_num_list):
    train_transformed, test_transformed, covariates = get_sample(num)
    rnn_model = RNNModel(
            model="LSTM",
            hidden_dim=30,
            dropout= 0.29909326760922184,
            batch_size=128,
            n_epochs=100,
            optimizer_kwargs={"lr": 1e-3},
            model_name="rnn_sales",
            random_state=42,
            training_length=30,
            input_chunk_length=28,
            force_reset=True,
            save_checkpoints=True,
            pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0],  
            "enable_progress_bar": False}
    )
    mape_score, rmse_score = evaluate_covariates_model(rnn_model, train_transformed, test_transformed, covariates )
    smape_list.append(mape_score)
    rmspe_list.append(rmse_score)


  0%|          | 0/100 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  1%|          | 1/100 [00:25<41:22, 25.07s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  2%|▏         | 2/100 [00:51<42:13, 25.

In [68]:
res_h2 = pd.DataFrame(
    {
        'rmspe' : rmspe_list,
        'smape' : smape_list,
    }
)
res_h2['Store'] = store_num_list
res_h2

,rmspe,smape,Store
0,0.096617,8.537655,10
1,0.062003,5.227861,62
2,0.058157,4.356692,110
3,0.110809,7.747403,163
4,0.063912,4.945696,300
...,...,...,...
95,0.056104,4.791607,960
96,0.068267,5.762917,1010
97,0.038038,3.219110,1046
98,0.083961,7.677319,1089


In [69]:
result = pd.merge(res_h1, res_h2, left_on='Store', right_on='Store')
result = result.set_index('Store')
result = pd.merge(result, store[['Store','StoreType']], left_on='Store', right_on='Store')
result.to_csv('../output/rnn_hyper_result.csv')
result.mean()

Store      585.390000
rmspe_x      0.076045
smape_x      6.459042
rmspe_y      0.086656
smape_y      7.037758
dtype: float64

In [70]:
result.groupby('StoreType').mean()

,Store,rmspe_x,smape_x,rmspe_y,smape_y
StoreType,,,,,
a,617.333333,0.072249,6.039484,0.082313,6.875778
b,528.600000,0.140178,11.943553,0.196384,14.794481
c,605.133333,0.068919,5.822658,0.074484,6.128657
d,552.633333,0.065591,5.686814,0.066595,5.523264
